In [1]:
import torch
import mymodel
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable 
import pickle  
import MeCab
from mecab_split import mecabsplit
tagger = MeCab.Tagger()

In [2]:
content_vocab = {'unk':0}
intent_vocab={}
intent_list=[]

data_intent=''
intent_idx=0
vocab_idx=1

for line in open('mydata.txt','r',encoding='utf-8'):
    line = line.strip().split('\t')
    if len(line)>1:
        intent=line[1]
        if intent not in intent_vocab:
            intent_vocab[intent]=intent_idx
            intent_list.append(intent)
            intent_idx +=1
    else:
        line = mecabsplit(tagger,line[0],False)
        for it in line:
            if it not in content_vocab:
                content_vocab[it] = vocab_idx # it : 단어
                vocab_idx +=1
                
                

In [3]:
cnn = mymodel.CNN_Text(vocab_idx,intent_idx)
print(vocab_idx, intent_idx) # 파일에 나타난 단어수, 의도 갯수(클래스)
optimizer = torch.optim.Adam(cnn.parameters())
cnn.train()    # train mode로 준비

43 2


CNN_Text(
  (embed): Embedding(43, 100)
  (convs1): ModuleList(
    (0): Conv2d(1, 20, kernel_size=(1, 100), stride=(1, 1))
    (1): Conv2d(1, 20, kernel_size=(2, 100), stride=(1, 1))
    (2): Conv2d(1, 20, kernel_size=(3, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.2)
  (fc1): Linear(in_features=60, out_features=2, bias=True)
)

In [4]:
epoch = 10
for e in range(epoch):
    totalloss = 0
    for line in open('mydata.txt','r',encoding='utf-8'): # 패턴 파일의 line별로 처리
        line = line.strip().split('\t')
    
        if len(line)> 1:
            target = Variable(torch.LongTensor([intent_vocab[line[1]]])) 
                     # LongTensor는 정수값 텐서
            continue

        optimizer.zero_grad()

        cont = []
        line = mecabsplit(tagger,line[0],False)
        for it in line:
            cont.append(content_vocab[it])
        cont = Variable(torch.LongTensor(cont)).view(1,-1)
        pred = cnn(cont)
        
        loss = F.cross_entropy(pred,target)
        totalloss += loss.data[0]
        loss.backward()  # 필터, 임베딩벡터 값, FC 레이어의 matrix 등 모든 값을 loss를 줄이는 방향으로 계산
        optimizer.step() # 벡터값을 변경함
    print (totalloss)

15.614544580595066
0.6688970052032346
0.08486135498628755
0.07373530421537566
0.04899213710632466
0.050210166377851806
0.029921369127933417
0.01791420551213463
0.00924291302429836
0.010030436675130261


In [5]:
response = []
for line in open('response.txt','r',encoding='utf-8'):
    line=line.strip()
    response.append(line)

In [6]:
for line in open('testdata.txt','r',encoding='utf-8'):
    line = line.strip()
    
    line = mecabsplit(tagger,line,False)
    cont = []
    for it in line:
        if it in content_vocab:
            cont.append(content_vocab[it])  # cont에는 입력문에 나타난 단어들의 index 저장됨
        else:
            cont.append(content_vocab['unk'])
    cont = Variable(torch.LongTensor(cont)).view(1,-1)
    pred = cnn(cont)
    v,i = torch.max(pred,1) # pred는 (p1, p2) 즉, 클래스별 확률 v: 둘중 큰값 i:큰값 클래스의 인덱스
    
    print('input : ',line)
    print('intent : ',intent_list[int(i)])
    print(response[int(i)])
    print()

input :  ['해외', '송금', '하', '려는데', '수수', '료', '얼마', '이', '죠', '?']
intent :  해외송금수수료
해외 송금시 수수료는 미화 2천달러 이하는 10,000입니다. 자세한 내용은 고객상담센터로 문의하세요.

input :  ['이체한도', '얼마', '이', 'ᆫ지', '조회', '하', 'ᆯ래요']
intent :  이체한도조회
이체한도 조회 및 변경은 홈페이지에서 하실 수 있습니다.

